<a href="https://colab.research.google.com/github/yamenka/UniRank_2021/blob/main/uniRankYamen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!{sys.executable} -m pip install pycaret


In [180]:
import pycaret
import pandas as pd
from pycaret.clustering import *
from pycaret.regression import *
from pycaret.utils import check_metric

In [182]:
df = pd.read_csv('https://github.com/Ainagbolahan/HDSC-21-SCIPY-GROUP/blob/main/dataset/wur_2021.csv?raw=true')
df2 = pd.read_csv('https://github.com/Ainagbolahan/HDSC-21-SCIPY-GROUP/blob/main/dataset/times-higher-edu-2021.csv?raw=true')

In [183]:
df2.pop('Overall')

0            95.6
1            94.9
2            94.8
3            94.5
4            94.4
          ...    
1521    10.3–25.0
1522    10.3–25.0
1523    10.3–25.0
1524    10.3–25.0
1525    10.3–25.0
Name: Overall, Length: 1526, dtype: object

In [184]:
new = df['Female:Male Ratio'].str.split(':',n=1, expand=True)
df['Male'] = new[1]
df['Female'] = new[0]


In [185]:
df.pop('Female:Male Ratio')

0       46 : 54
1       44 : 56
2       49 : 51
3       36 : 64
4       39 : 61
         ...   
1521    42 : 58
1522    57 : 43
1523    54 : 46
1524    59 : 41
1525    34 : 66
Name: Female:Male Ratio, Length: 1526, dtype: object

In [186]:
df['International Students'].replace('%',value='',regex=True,inplace=True)

In [187]:
df['International Students'] = pd.to_numeric(df['International Students'])

In [188]:
df['No. of FTE Students'].replace(',',value='',regex=True, inplace = True)

In [189]:
df['No. of FTE Students'] = pd.to_numeric(df['No. of FTE Students'])

In [190]:
df['Female'] = pd.to_numeric(df['Female'])
df['Male'] = pd.to_numeric(df['Male'])
df3 = df.merge(df2,how='outer',on='NameCountry/Region')


In [191]:
df3.pop('Rank_y')

0              1
1              2
2              3
3              4
4              5
          ...   
1523       1001+
1524       1001+
1525       1001+
1526     301–350
1527    801–1000
Name: Rank_y, Length: 1528, dtype: object

In [200]:
df3['Rank_x'] = df3['Rank_x'].str.replace('=', '')
df3['Rank_x'] = df3['Rank_x'].str.replace('+', '')
new3 = df3['Rank_x'].str.split('–', n=1, expand=True)
df3['Rank_x'] = new3[0]
df3['Rank_x'] = pd.to_numeric(df3['Rank_x'])


In [202]:
dataset = df3

In [204]:
data = dataset.sample(frac=0.9,random_state=786)
data_unseen = dataset.drop(data.index)
data.reset_index(drop=True,inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

In [205]:
exp_reg101 = setup(data = data, ignore_features=['NameCountry/Region'], target='Rank_x',session_id=123)

,Description,Value
0,session_id,123
1,Target,Rank_x
2,Original Data,"(1375, 12)"
3,Missing Values,True
4,Numeric Features,10
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(961, 9)"


In [206]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,24.5816,1.739319e+03,41.4302,0.9838,0.0990,0.0642,0.456
lightgbm,Light Gradient Boosting Machine,31.1000,2.391757e+03,48.6056,0.9779,0.1801,0.0994,0.135
gbr,Gradient Boosting Regressor,33.7297,2.585095e+03,50.5168,0.9761,0.1477,0.1021,0.170
rf,Random Forest Regressor,30.2856,2.599725e+03,50.5401,0.9759,0.1303,0.0858,0.599
ada,AdaBoost Regressor,59.1304,5.105629e+03,71.1081,0.9529,0.3739,0.4848,0.124
dt,Decision Tree Regressor,35.1091,5.943321e+03,75.8158,0.9449,0.1968,0.1075,0.019
lr,Linear Regression,67.6153,7.923390e+03,88.6993,0.9271,0.4960,1.2853,0.277
lasso,Lasso Regression,67.6184,7.920485e+03,88.6832,0.9271,0.4955,1.2845,0.013
ridge,Ridge Regression,67.6154,7.923388e+03,88.6993,0.9271,0.4960,1.2853,0.013
lar,Least Angle Regression,67.6153,7.923389e+03,88.6993,0.9271,0.4960,1.2853,0.014
